In [17]:
import numpy as np

from keras.models import Model
from keras.layers import Embedding, Reshape, Activation, Input
from keras.layers.merge import Dot
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import skipgrams


import codecs
import time
np.random.seed(13)


In [18]:
path = '/home/junsoo/PycharmProjects/word2vec_sample/text8'
path = '/home/junsoo/PycharmProjects/word2vec_sample/text_sample'
corpus = codecs.open(path, "r", encoding='utf-8', errors='ignore').read()
words = corpus.split()
corpus = []
sentence = ''


In [19]:
# word_count_per_sentence = 100
word_count_per_sentence = 20

for idx,word in enumerate(words):
    idx += 1
    sentence += word + ' '
    if idx % word_count_per_sentence == 0:
        corpus.append(sentence.strip())
        print(sentence+'\n')
        sentence = ''


tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\r\t\n‘“')
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %s' % vocab_size)


anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english 

revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to 

describe any act that used violent means to destroy the organization of society it has also been taken up as 

a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king 

anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing 

interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly 

the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a 

harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive econo

In [20]:
dim_embedddings = 128

# inputs
word_inputs = Input(shape=(1,), dtype='int32')
word_vector = Embedding(vocab_size, dim_embedddings, embeddings_initializer='glorot_uniform')(word_inputs)

# context
context_inputs = Input(shape=(1,), dtype='int32')
context_vector = Embedding(vocab_size, dim_embedddings, embeddings_initializer='glorot_uniform')(context_inputs)

output_layer = Dot(axes=2)([word_vector, context_vector])
output_layer = Reshape((1,), input_shape=(1, 1))(output_layer)
output_layer = Activation('sigmoid')(output_layer)

SkipGram = Model(inputs=[word_inputs, context_inputs], outputs=output_layer)
SkipGram.summary()
SkipGram.compile(loss='binary_crossentropy', optimizer='Nadam', metrics=['accuracy'])


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 1)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 1, 128)        13440       input_5[0][0]                    
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 1, 128)        13440       input_6[0][0]                    
___________________________________________________________________________________________

In [21]:
epochs = 2
t2s = tokenizer.texts_to_sequences(corpus)
len_t2s = len(t2s)


In [23]:
for cur_epoch in range(epochs):
    loss = 0.
    accuracy = 0.

    start_time = time.time()
    for i, doc in enumerate(t2s):
        data, labels = skipgrams(sequence=doc, vocabulary_size=vocab_size, window_size=4, negative_samples=5.)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if x:
            train_result = SkipGram.train_on_batch(x, y)
            loss += train_result[0]
            accuracy += train_result[1]

        # if i % 10000 == 0:
        #     print("\t%d/%d: %s\t%s" % (i + 1, len_t2s, loss, accuracy))

    avg_loss = loss / len_t2s
    avg_acc = accuracy / len_t2s
    end_time = time.time()
    duration = end_time - start_time

    print("\t%d/%d: %s\t%s\t[%f sec]" % (cur_epoch+1, epochs, avg_loss, avg_acc, duration))


	1/2: 0.686061151326	0.56696433574	[0.049660 sec]
	2/2: 0.678839303553	0.616666741669	[0.061425 sec]


In [24]:
print("Save weights...")
vector_filename = 'vectors_notebook.txt'
f = open(vector_filename ,'w')
f.write('{} {}\n'.format(vocab_size - 1, dim_embedddings))
vectors = SkipGram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
f.close()


Save weights...
